In [92]:
import pandas as pd
from SentiCR.SentiCR import SentiCR

## Preprocessing

In [93]:
filename = '/Users/erdemdemir/DataspellProjects/empiricalSE/data/golang/Aug2020-Sep2020/golang_Aug2020-Sep2020.xml.out'
df_original = pd.read_xml(filename)

In [94]:
df = df_original[4:].drop(['team_domain', 'channel_name', 'start_date', 'end_date'], axis=1)
df

,conversation_id,ts,user,text
4,1.0,2020-08-01T00:01:15.595000,Samarie,{{ with .Data }} also works
5,1.0,2020-08-01T00:01:22.227000,Samarie,if .Data is invalid it won't be rendered
6,1.0,2020-08-01T00:02:50.586000,Cephas,ah
7,1.0,2020-08-01T00:02:51.116000,Cephas,true
8,1.0,2020-08-01T00:02:55.357000,Cephas,okay okay thank you
...,...,...,...,...
60078,1915.0,2020-09-30T18:06:13.071000,Cristofor,then ill check freenode later
60079,1915.0,2020-09-30T18:16:02.925000,Samarie,I unironically prefer IRC over Slack
60080,1914.0,2020-09-30T18:24:10.424000,Cristofor,as long as its not in the browser me too
60081,1915.0,2020-09-30T18:24:27.760000,Cristofor,i made my GUI build unchanged for Android! 😄


### SentiCR preprocessing

In [95]:
import numpy as np
from sklearn.metrics import classification_report
import time
sentiment_analyzer=SentiCR()
def predict_sentiCR(df, column = 'first_message'):
    begin=time.time()
    sentences=df[column]
    pred=[]
    for sent in sentences:
        score=sentiment_analyzer.get_sentiment_polarity(sent)
        pred.append(score)
    
    end=time.time()
    print('Prediction used {:.2f} seconds'.format(end-begin))
    return pred

StackOverflow Reading data from oracle..
Training classifier model..


/Users/erdemdemir/DataspellProjects/bert-arxiv/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ani', 'continu', 'deleg', 'doe', 'doubl', 'dure', 'els', 'endwhil', 'extend', 'implement', 'includ', 'interfac', 'namespac', 'nativ', 'nowwhil', 'onc', 'ourselv', 'overrid', 'packag', 'privat', 'protect', 'rais', 'readon', 'requir', 'sign', 'synchron', 'themselv', 'tri', 'veri', 'yourselv'] not in stop_words.
  warnings.warn(


Training used 4.96 seconds


In [96]:
from os.path import exists
sentiCR_path = filename[-30:-8]+'-CR.csv'

if exists(sentiCR_path):
    df_sentiCR = pd.read_csv(sentiCR_path)
else:
    pred_full = predict_sentiCR(df, 'text')
    df_sentiCR = df.copy()
    df_sentiCR['sentiCR'] = Extract(pred_full)
    df_sentiCR['sentiCR'] = df_sentiCR['sentiCR'].replace({2: -1})
    df_sentiCR.to_csv(sentiCR_path)

In [97]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(pearsonr(tmp[r], tmp[c])[1], 4)
    return pvalues

### Senti4SD preprocessing
For the preprocessing of Senti4SD we do the sentiment score calculation outside this notebook file. Make sure yuo save the csv to predictions_senti4SD.csv

In [98]:
df_senti4SD = df.copy()
df_senti4SD = df_senti4SD.drop(['ts', 'conversation_id', 'user'], axis=1).rename(columns={"text": "Text"})
df.index.name = 'ID'
print('Use this dataframe to feed to the senti4SD repo')
senti4SD_path = filename[-30:-8]+'-4SD.csv'
df_senti4SD.to_csv(senti4SD_path, sep=';')
df_senti4SD.head()

Use this dataframe to feed to the senti4SD repo


,Text
4,{{ with .Data }} also works
5,if .Data is invalid it won't be rendered
6,ah
7,true
8,okay okay thank you


In [99]:
senti4SD = pd.read_csv(senti4SD_path).sort_values(by=['ID']).set_index('ID')
senti4SD['PREDICTED'] = senti4SD['PREDICTED'].replace({'negative': -1, 'neutral': 0, 'positive': 1})

ParserError: Error tokenizing data. C error: Expected 1 fields in line 152, saw 2


In [ ]:
senti4SD_df = df.copy()
senti4SD_df['senti4SD'] = senti4SD['PREDICTED'].copy()
senti4SD_df.to_csv(senti4SD_path)
senti4SD_df.head()